<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/multiplicacion_matrices_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se multiplicara dos matrices A y B, el resultado de la multiplicacion se guardara en la matriz de resultado C
C=AB\begin{pmatrix}a_{11}&\cdots &a_{1n}\\\vdots &\ddots &\vdots \\a_{m1}&\cdots &a_{mn}\end{pmatrix}\begin{pmatrix}b_{11}&\cdots &b_{1p}\\\vdots &\ddots &\vdots \\b_{n1}&\cdots &b_{np}\end{pmatrix}\begin{pmatrix}a_{11}b_{11}+\cdots +a_{1n}b_{n1}&\cdots &a_{11}b_{1p}+\cdots +a_{1n}b_{np}\\\vdots &\ddots &\vdots \\a_{m1}b_{11}+\cdots +a_{mn}b_{n1}&\cdots &a_{m1}b_{1p}+\cdots +a_{mn}b_{np}\end{pmatrix}


EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y el manejo de la operacion de matriz a bajo nivel. El ejemplo es ilustrativo para entededer los multi hilos de la GPU en dos dimenciones.

---
# 2 Armado del ambiente
Toma la direcciòn web de una imagen con  acceso público en internet, la deja disponible al contexto de ejecuciòn del cuaderno colab.

In [67]:
#@title # 2.1 Parámetros de ejecución
#@markdown ### Especifique las dimeciones de la matrices:

try: 
  fila_mat_a =  250#@param {type:"integer"}
  colum_mat_a =  250#@param {type:"integer"}

  fila_mat_b =  250#@param {type:"integer"}
  colum_mat_b =  250#@param {type:"integer"}

except:
  print("Error de ingresar los parametros")








Validar las matrices

In [68]:
#A.B
if fila_mat_a < 0 or colum_mat_a < 0:
  print("ERROR LAS DIMENCIONES DE LA MATRIZ A TIENE QUE SER MAYOR A 0")
if fila_mat_b < 0 or colum_mat_b < 0:
  print("ERROR LAS DIMENCIONES DE LA MATRIZ B TIENE QUE SER MAYOR A 0")
if colum_mat_a != fila_mat_b:
  print("ERROR LAS DIMENCIONES DE LAS MATRIZ NO PERMITEN REALIZAR LA MULTIPLICACION")

---
## 2.2 Instala en el cuaderno el módulo CUDA de Python.

In [55]:
!pip install pycuda

---
# 3 Desarrollo
Ejecución del algoritmo escala de grises en GPU.

In [69]:
%matplotlib inline
from datetime import datetime
tiempo_total = datetime.now()

import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from   pycuda.compiler import SourceModule

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

tiempo_definicion_matrices = datetime.now()

# CPU - Defino la memoria de la matriz A en cpu.
matriz_a = numpy.random.randn(fila_mat_a, colum_mat_a)*10
matriz_a = matriz_a.astype(numpy.int32())

# CPU - Defino la memoria de la matriz B en cpu.
matriz_b = numpy.random.randn(fila_mat_b, colum_mat_b)*10
matriz_b = matriz_b.astype(numpy.int32())

# CPU - Defino la memoria de la matriz resultado C en cpu.
matriz_c = numpy.random.randn(fila_mat_a, colum_mat_b)*0
matriz_c = matriz_c.astype(numpy.int32())
matriz_c_secuencial = numpy.random.randn(fila_mat_a, colum_mat_b)*0
matriz_c_secuencial = matriz_c_secuencial.astype(numpy.int32())

tiempo_definicion_matrices = datetime.now() - tiempo_definicion_matrices

tiempo_multiplicacion_secuencial = datetime.now()

for f in range(fila_mat_a):
  for c in range(colum_mat_b):
    for inter in range(colum_mat_a):
      matriz_c_secuencial[f][c] += matriz_a[f][inter]*matriz_b[inter][c]

tiempo_multiplicacion_secuencial = datetime.now() - tiempo_multiplicacion_secuencial
    

tiempo_reserva_memoria_GPU = datetime.now()
# CPU - reservo la memoria GPU.
matriz_a_Gpu = cuda.mem_alloc(matriz_a.nbytes)
matriz_b_Gpu = cuda.mem_alloc(matriz_b.nbytes)
matriz_c_Gpu = cuda.mem_alloc(matriz_c.nbytes)
tiempo_reserva_memoria_GPU = datetime.now() - tiempo_reserva_memoria_GPU

tiempo_copia_memoria_GPU = datetime.now()
# GPU - Copio la memoria al GPU.
cuda.memcpy_htod(matriz_a_Gpu, matriz_a)
cuda.memcpy_htod(matriz_b_Gpu, matriz_b)
cuda.memcpy_htod(matriz_c_Gpu, matriz_c)
tiempo_copia_memoria_GPU = datetime.now() - tiempo_copia_memoria_GPU

#CPU - Defino la funcion kernel que ejecutará en GPU
module = SourceModule("""
__global__ void multiplicar(int ancho, int alto, int inter, int *matriz_a , int *matriz_b, int *matriz_c)
{
    // Calculo las coordenadas del Thread en dos dimensiones.
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    int idy = threadIdx.y + blockIdx.y*blockDim.y;
    
    // Verifico que los Thread, esten dentro de las dimensiones de la imagen.
    if( idx < ancho && idy < alto ) {
      int indice = idy+(idx*ancho);
      int i = 0;
      int resul = 0;

      //
      while (i<inter) {
        //multiplico fila por columna y sumo el resultado de cada componente
        resul += matriz_a[idx*inter + i]*matriz_b[idy + i*inter];
        i++;
      }
      //paso el resultado obtenido a la componente de la matriz
      matriz_c[indice] = resul;
    }
}

""")

# CPU - Genero la función kernel.
kernel = module.get_function("multiplicar")

dim_hilo_x = 16
dim_bloque_x = numpy.int( (fila_mat_a+dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 19
dim_bloque_y = numpy.int( (colum_mat_b+dim_hilo_y-1) / dim_hilo_y )

print( "Thread: [", 
      dim_hilo_x, 
      ",", 
      dim_hilo_y, 
      " ], Bloque : [", 
      dim_bloque_x, 
      ",", 
      dim_bloque_y, 
      "]" )
print( "Total de Thread: [", 
      dim_hilo_x*dim_bloque_x, 
      ",", 
      dim_hilo_y*dim_bloque_y, 
      " ]", 
      " = ", 
      dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

tiempo_multiplicacion_paralela = datetime.now()

kernel( numpy.int32(fila_mat_a), 
        numpy.int32(colum_mat_b), 
        numpy.int32(colum_mat_a), 
        matriz_a_Gpu, 
        matriz_b_Gpu, 
        matriz_c_Gpu, 
        block=( dim_hilo_x, dim_hilo_y, 1 ), 
        grid=(dim_bloque_x, dim_bloque_y,1) )

tiempo_multiplicacion_paralela = datetime.now() - tiempo_multiplicacion_paralela

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( matriz_c, matriz_c_Gpu )
tiempo_total = datetime.now() - tiempo_total

print("Matiz A:\n", matriz_a)
print("Matiz B:\n", matriz_b)
print("Matiz C Secuencial:\n", matriz_c_secuencial)
print("Matiz C Paralela:\n", matriz_c)
print("Matiz C Paralela:\n", matriz_c)
print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU  : ", tiempo_en_ms( tiempo_multiplicacion_paralela ), "[ms]" )
print("Tiempo CPU  : ", tiempo_en_ms( tiempo_multiplicacion_secuencial ), "[ms]" )

Thread: [ 16 , 19  ], Bloque : [ 16 , 14 ]
Total de Thread: [ 256 , 266  ]  =  68096
Matiz A:
 [[ 16   3   9 ...   0  11  -7]
 [-13 -18  -1 ...   4 -12  -7]
 [ 23  10 -28 ...  -8 -11  -3]
 ...
 [ -5   3   3 ...  -4   1  18]
 [ -3   0  -6 ...   2  -3  -4]
 [-16 -18   9 ... -15 -10   6]]
Matiz B:
 [[  0 -20   0 ...   5  -2 -12]
 [-20  -3   6 ...  15   4  -3]
 [ -2  -7   0 ... -17   3  15]
 ...
 [ -5  -5   0 ...   9   9 -12]
 [ -7  -5  11 ...   0  16   7]
 [-25  -6 -19 ...   6  -6   3]]
Matiz C Secuencial:
 [[ 1544  2443 -1319 ... -1589  -978 -2111]
 [ 1006  1761 -1308 ... -2460  -101   769]
 [ 1319 -1330  -352 ...  1156 -1162  1462]
 ...
 [   46   126  -237 ... -1745   952  -956]
 [  336    75  -562 ...   523   574   400]
 [  111  -801 -1542 ...   158 -1081   300]]
Matiz C Paralela:
 [[ 1544  2443 -1319 ... -1589  -978 -2111]
 [ 1006  1761 -1308 ... -2460  -101   769]
 [ 1319 -1330  -352 ...  1156 -1162  1462]
 ...
 [   46   126  -237 ... -1745   952  -956]
 [  336    75  -562 ...   523 

---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  wget url_imagen       | Lectura de la direcciòn URL de la imagen (jpg) a procesar.
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  matplotlib inline    | Macro de Colab para mostrar imagenes.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  Image.open()          | Abre el archivo de la imagen.
CPU      |  numpy.asarray(imagen) | Convierte el formato comprimido JPG a RAW.
CPU      |  numpy.empty_like(()   | Genera el array destino, que tendrá a la imagen resultado. 
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las imagenes en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las imagenes al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x, dim_hilo_y| Calcula las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa los atributos de la imagen.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.
CPU      |  plt.imshow            | Muestra la imagen original.
CPU      |  plt.imshow            | Muestra la imagen resultado.



---
# 5 Conclusiones

Las conclusiones son explicadas en clase...

---
# 6 Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[4] 2009, SINTESIS DIGITAL DE COLOR UTILIZANDO TONOS DE GRIS, ING. JESÚS GUSTAVO FLORES ERAÑA : [PDF](https://ninive.uaslp.mx/xmlui/bitstream/handle/i/2264/MCA1SDC00901.pdf?sequence=1&isAllowed=y)
